We can convert data from its tsv from to actual usable files that we can directly feed our pipelines.

! Always provide tsv file as input for the repo https://github.com/cambridgeltl/MTL-Bioinformatics-2016/tree/master/data

In [1]:
import json

In [ ]:
dataset = []

# Open the file and read line by line
# give a path of your locally saved tsv file validation data
with open(r"G:\GitClone\BioNER_Project\Gen2FT\BC2GM\devel.tsv", "r", encoding="utf-8") as file:
    current_sentence = []
    for line in file:
        line = line.strip()
        if line:  # Check if the line is not empty
            
            token, tag = line.split('\t')
            
            current_sentence.append(f"{token}\t{tag}")
        else:  # End of sentence
            # current_sentence.append('.')  # Add period to denote end of sentence
            dataset.extend(current_sentence)
            current_sentence = []

print("First file length",len(dataset))
# since train dataset was not sufficient i m merging previous dataset into it 
# give a path of your locally saved tsv file test data
dataset2=dataset
with open(r"G:\GitClone\BioNER_Project\Gen2FT\new _dataset\bc2gm gene\test.tsv", "r", encoding="utf-8") as file:
    current_sentence = []
    for line in file:
        line = line.strip()
        if line:  # Check if the line is not empty
            
            token, tag = line.split('\t')
            
            current_sentence.append(f"{token}\t{tag}")
        else:  # End of sentence
            # current_sentence.append('.')  # Add period to denote end of sentence
            dataset.extend(current_sentence)
            current_sentence = []
dataset.extend(dataset2)

print("Second file length",len(dataset))
# since train dataset was not sufficient i m merging previous dataset into it.
# give a path of your locally saved tsv file train data
dataset2=dataset
with open(r"G:\GitClone\BioNER_Project\Gen2FT\new _dataset\bc2gm gene\devel.tsv", "r", encoding="utf-8") as file:
    current_sentence = []
    for line in file:
        line = line.strip()
        if line:  # Check if the line is not empty
            
            token, tag = line.split('\t')
            
            current_sentence.append(f"{token}\t{tag}")
        else:  # End of sentence
            # current_sentence.append('.')  # Add period to denote end of sentence
            dataset.extend(current_sentence)
            current_sentence = []
dataset.extend(dataset2)
print("Third file length",len(dataset))


Merging will not cause any problem even after merging I m going to split them into 80 20 ratio for train and test.

In [ ]:

#----------------------------------------------------Processing starts here
processed=[]

sen=[]
tag=[]
for item in dataset:
    if "." not in item:
        sen.append(item.split("\t")[0])
        tag.append(item.split("\t")[-1])
    else: 
        processed.append({"sentence":" ".join(sen),"tokens":sen,"tags":tag})
        sen=[]
        tag=[]

final=[p for p in processed if "B-GENE" in p['tags'] or "I-GENE" in p['tags']]
final2=[p for p in final if len(p['tokens'])<130] # token size set to 130 max
len(final2)
import pandas as pd
rawdf=pd.DataFrame(final2)
import re

# Define a function to generate ner_tags
def generate_ner_tags(tokens, tags):
    ner_tags = []
    for token, tag in zip(tokens, tags):
        if tag != 'O':
            # Check if the token is non-alphanumeric and single letter
            if not (token.isalnum() and len(token) > 1):
                continue
            ner_tags.append(f"{tag}:{token}")
    return ner_tags

# Convert DataFrame to the desired format
formatted_data = []
for idx, row in rawdf.iterrows():
    tokens = row['tokens']
    sentence = row['sentence']
    tags = row['tags']
    ner_tags = generate_ner_tags(tokens, tags)
    gemma_prompt=f'\nInstruction:\nDo NER on this sentence."{sentence}"\nReply in key value pair as Tag and Entity.'
    llama_prompt=f'''<s>[INST]Do NER on this sentence.\n"{sentence}"\nReply in key value pair as Tag and Entity.[/INST]'''
    response=f'\n\nResponse:\n{ner_tags}'
    formatted_data.append({
        'tokens': tokens,
        'text': sentence,
        'ner_tags': ner_tags,
        'tags': tags,
        'gemma_prompt':gemma_prompt,
        'llama_prompt':llama_prompt,
        'response': response,
        'llama_response': response+"</s>"
    })

train_split=int(len(formatted_data)*.80)
val_split=int(len(formatted_data)*.20)



In [ ]:
# Save the formatted data to a JSON file
# provide a local path where to save the files created in json
train_file = r"G:\GitClone\BioNER_Project\Gen2FT\BC2GM\bc2gm_gene_dataset_train.json"
with open(train_file, 'w') as f:
   json.dump(formatted_data[:train_split], f, indent=4)

# provide the path to save test file in json
validation_file = r"G:\GitClone\BioNER_Project\Gen2FT\BC2GM\bc2gm_gene_dataset_validation.json"
with open(validation_file, 'w') as f:
   json.dump(formatted_data[-val_split:], f, indent=4)


print("Data saved to", train_file, validation_file)